In [1]:
### source: https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Object-Detection

- **Single-Shot Detection.** Earlier architecture for object detection consisted of two distinct stages - a region proposal network that performs object localization and a classifier for detecting the types of objects in the proposed regions. Computationally, these can be very expensive and therefore ill-suited for real-world, real-time applications. Single-shot models encapsulate both localization and detection tasks in a single forward sweep of the network, resulting in significantly faster detections while deployable on lighter hardware
- **Multiscale Feature maps.** In image classification tasks, we base our predictions on the final convolutional feature map - the smallest but deepest representation of the original image. In object detection, feature maps from intermediate convolutional layers can also be directly useful because they represent the original image at different scales. Therefore, a fixed-size filter operating on different feature maps will be able to detect objects of various sizes.
- **Priors.** These are pre-computed boxes defined at specific positions on specific feature maps, with specific aspect ratios and scales. They are carefully chosen to match the characteristics of objects bounding boxes (i.e. the ground truths) in the dataset.
- **Multibox.**  This is a technique that formulates predicting an object's bounding box as a regression problem, wherein a detedted object's coordinates are regressed to its ground truth's coordinates. In addition, for each predicted box, scores are generated for various object types. Priors serve as feasible starting points for predictions because they are modeled on the ground truths. Therefore, there will be as many predicted boxes as there are priors, most of whom will contain no object.
- **Hard Negative Minig.** This refers to explicitly choosing the most egregious false positives predicted by a model and forcing it to learn from these examples. In other words, we are mining only those negatives that the model found hardest to identify correctly. In the context of object detection, where the vast majority of predicted boxes do not contain an object, this also serves to reduce the negative-positive imbalance.
- **Non-Maximum Suppression.** At any given location, multiple priors can overlap significantly. Therefore, predictions arising out of these priors could actually be duplicates of the same object. Non-Maximum Suppression (NMS) is a means to remove redundant predictions by suppressing all but the one with the maximum score.

#### Some definitions
- **Boundary coordinates.** The most obvious way to represent a box is by the pixel coordinates of the x and y lines that constitute its boundaries. But pixel values are next to useless if we don't know the actual dimensions of the image. A better way would be to represent all coordinates is in their fractional form.
- **Center-Size coordinates.** This is a more explicit way of representing a box's position and dimensions. In the code, you will find that we routinely use both coordinates systems depending upon their suitability for the task, and always in their fractionl forms.
- **Jaccard Index.** The Jaccard Index or Jaccard Overlap or Intersection-over-Union (IoU) measure the degree or extent to which two boxes overlap. An IoU of 1 implies that they are the same box, while a value of 0 indicates they're mutually exclusive spaces.

#### Multibox
Is a technique for detecting objects where a prediction consists of two components:
- Coordinates of a box that may or may not contain an object. This is a regression task.
- Scores for various object types for this box, including a background class which implies there is no object in the box. This is a classification task.

#### Single Shot Detector (SSD)
Is a purelly convolutional neural network (CNN) that we can organize into three parts:
- Base convolutions derived from an existing image classification architecture that will provide lower-level feature maps.
- Auxiliary convolutions added on top of the base network that will provide higher-level feature maps.
- Prediction convolutions that will locate and identify objects in these feature maps.

The paper demonstrates two variants of the model called the SSD300 and SSD512. The suffixes represent the size of the input image. Although the two networks differ slightly in the way they are constructed, they are in principle the same. For convenience, we will deal with SSD300

#### Base Convolutions - part 1
First of all, why use convolutions from an existing network architecture? Because models proven to work well with image classification are already pretty good at capturing the basic essence of an image. The convolutional features are useful for object detection, albeit in a more local sense - we're less interested in the image as a whole than specific regions of it where objects are present.

There's also the added advantage of being able to use layers pretrained on a reliable classification dataset. As your may know, this is called Transfer Learning. By borrowing knowledge from a different but closely related task, we've made progress before we've even begun.

The authors of the paper employ the VGG-16 architecture as their base network. It's rather simple in ints original form.

They recommend using one that's pretrained on the ImageNet Large Scale Visual Recognition Competition (ILSVRC) classification task. Luckily, there's one already available in PyTorch, as are other popular architectures. If you wish, you could opt for something larger like the ResNet. Just be mindful of the computational requirements.

As per the paper, we've to make some changes to this pretrained network to adapt it to our own challenge of object detection. Some are logical and necessary, while others are mostly a matter of convenience or preference.

- The input image size will be 300, 300, as stated earlier.
- The 3rd pooling layer, which halves dimensions, will use the mathematical ceiling function instead of the default floor function in determining output size. This is significant only if the dimentions of the preceding feature map are odd and not even. By looking at the image above, you could calculate that for our input image size of 300, 300, the conv3_3 feature map will be of cross-section 75, 75, which is halved to 38, 38 instead of an inconvenient 37, 37.
- We modify the 5th pooling layer from a 2, 2 kernel and 2 stride of a 3, 3 kernel and 1 stride. The effect this has is it no longer halves the dimentions of the feature map from the preceding convolutional layer.
- We don't need the fully connected (i.e. classification) layers because they serve no purpose here. We will tossfc8 away completely, but choose to rework fc6 and fc7 into convolutional layers conv6 and conv7.

#### FC -> Convolutional layer
In the typical image classification setting, the first fully connected layer cannot operate on the preceding feature map or image directly. We'd need to flatten into a 1D structure. Therefore, any fully connected layer can be converted to an equivalent layer simply by reshaping its parameters.

#### Base Convolutions - part 2
(...)


#### A detour
Before we move on to the prediction convolutions, we must first understand what it is we are predicting. Sure, it's objects and their positions, but in what form? It is here that we must learn about priors and the crucial role they play in the SSD.

#### Priors
Object predictions can be quite diverse, and I don't just mean their type. They can occur at any position, with any size and shape. Mind you, we shouldn't go as far as to say there are infinite possibilities for where and how an object can occur. While this may be true mathematically, many options are simply improbable or uninsteresting. Furthermore, we needn't insist that boxes are pixel-perfect.

In effect we can discretize the mathematical space of potential predictions into just thousands of possibilities.

Priors are precalculated, fixed boxes which collectievly represent this universe of probable and approximate box predictions.

Priors are manually but carefully chosen based on the shapes and sizes of ground truth objects in our dataset. By placing these priors at every possible location in a feature map, we also account for variety in position.

In defining the priors, the authors specify that - 
- they will be applied to various low-level and high-level feature maps, viz. thos from conv4_3, conv7, conv8_2, conv9_2, conv10_2 and conv11_2. These are the same feature maps indicated on the figures before.
- if a prior has a scale s, then its area is equal to that of a square with side s. The largest feature map, conv4_3, will have priors with a scale of 0.1, i.e. 10% of image's dimensions, while the rest have priors with scales linearly increasing from 0.2 to 0.9. As you can see, larger feature maps have priors with smaller scales and therefore ideal for detecting smaller objects.
- at each position on a feature map, there will be priors of various aspect ratios. All feature maps will have priors with ratios 1:1, 2:1, 1:2. The intermediate feature maps of conv7, conv8_2 and conv9_2 will also have prios with ratios 3:1, 1:3. Moreover, all feature maps will have one extra prior with an aspect ratio of 1:1 and at a scale that is the geometric mean of the scales of the current and subsequent feature map.

#### Predictions vis-à-vis Priors
Earlier, we said we would use regression to find the coordinates of an object's bounding box. But then, surely, the priors can't represent our final predicted boxes? They don't

Again, I would like to reiterate that the priors represent, approximately, the possibilities for prediction.

This means that we use each prior as an approximate staring point and then find out how much it needs to be adjusted to obtain a more exact prediction for a bounding box.

So if each predicted bounding box is a slight deviation from a prior, and our goal is to calculate this deviation, we need a way to measure or quantify it.

Consider a cat, its predicted bounding box, and the prior with which the prediction was made. Assume they are represented in center-size coordinates, which we are familiar with.

This answers the question we posed at the beginning of this section. Considering that each prior is adjusted to obtain a more precise prediction, these four offsets are the form in which we will regress bounding boxes coordinates.

As you can see, each offset is normalized by the corresponding dimension of the prior. This makes sense because a certain offset would be less significant for a larger prior than it would be for a smaller prior.

#### Prediction convolutions
Earlier, we earmarked and defined priors for six feature maps of various scales and granularity, viz. thos from conv4_3, conv7, conv8_2, conv9_2, conv10_2, and conv11_2.

Then, for each prior at each location on each feature map, we want to predict - 
- the offsets (g_c_x, g_c_y, g_w, g_h) for a bounding box.
- a set of n_classes scores for the bounding box, where n_classes represents the total number of object types (including background class).

To do this in the simplest manner possible, we need two convolutional layers for each feature map - 
- a localization prediction convolutional layer with a 3, 3 kernel evaluating at each location (i.e. with padding and stride of 1) with 4 filters for each prior present at the location. The 4 filters for a prior calculate the four encoded offsets (g_c_x, g_c_y, g_w, g_h for the bounding box predicted from that prior).
- a class prediction convolutional layer with a 3, 3 kernel evaluating at each location (i.e. with padding and stride of 1) with n_classes filters for each prior present at the location. The n_classes filters for a prior calculate a set of n_classes scores for that prior.

All our filters are applied with a kernel size of 3, 3.

We don't really need kernels (or filters) in the same shapes as the priors because the different filters will learn to make predictions with respect to the different prior shapes.

What we're really interested in is the third dimension, i.e. the channels. These contain the actual predictions. The channel values at each position of the location predictions represent the encoded offsets with respect to the priors at that position. This is the final output of the prediction stage. A stack of boxes, if you will, and estimates for what's in them. It's all coming together, isn't it? If this is your first rodeo in object detection, I should think there's now a faint light at the end of the tunel.

#### Multibox loss